<a href="https://colab.research.google.com/github/HisakaKoji/sentence-transformers/blob/master/hisaka_test_20200323.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
%cd /content
!git clone https://github.com/sonoisa/sentence-transformers
!cd sentence-transformers; pip install -r requirements.txt
!wget -O sonobe-datasets-sentence-transformers-model.tar "https://www.floydhub.com/api/v1/resources/JLTtbaaK5dprnxoJtUbBbi?content=true&download=true&rename=sonobe-datasets-sentence-transformers-model-2"
!tar -xvf sonobe-datasets-sentence-transformers-model.tar

/content
fatal: destination path 'sentence-transformers' already exists and is not an empty directory.
--2020-03-23 01:43:15--  https://www.floydhub.com/api/v1/resources/JLTtbaaK5dprnxoJtUbBbi?content=true&download=true&rename=sonobe-datasets-sentence-transformers-model-2
Resolving www.floydhub.com (www.floydhub.com)... 104.26.0.30, 104.26.1.30, 2606:4700:20::681a:11e, ...
Connecting to www.floydhub.com (www.floydhub.com)|104.26.0.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/tar]
Saving to: ‘sonobe-datasets-sentence-transformers-model.tar’

sonobe-datasets-sen     [     <=>            ] 422.28M  33.6MB/s    in 13s     

2020-03-23 01:43:28 (32.5 MB/s) - ‘sonobe-datasets-sentence-transformers-model.tar’ saved [442788352]

./
./training_bert_japanese/
./training_bert_japanese/0_BERTJapanese/
./training_bert_japanese/0_BERTJapanese/added_tokens.json
./training_bert_japanese/0_BERTJapanese/config.json
./training_bert_japanese/0_BERTJ

In [57]:
%cd sentence-transformers

/content/sentence-transformers


In [0]:
%tensorflow_version 2.x
from sentence_transformers import SentenceTransformer
import numpy as np

model_path = "/content/training_bert_japanese"
model = SentenceTransformer(model_path, show_progress_bar=False)

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('input_spot.txt',header = None)



In [0]:
df = df.rename(columns={0: 'description'})

In [0]:
df_name = pd.read_csv('name_spot.txt',header = None)

In [0]:
df_name = df_name.rename(columns={0: 'name'})

In [0]:
df_comb = pd.concat([df_name,df], axis=1)

In [0]:
df_comb = df_comb.dropna()

In [66]:
df_comb.columns

Index(['name', 'description'], dtype='object')

In [0]:
sentences = df_comb['description'].values.tolist()

In [0]:
sentence_vectors = model.encode(sentences)

In [0]:
sentences_name = df_comb['name'].values.tolist()

In [0]:
from sklearn.cluster import KMeans

num_clusters = 8
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(sentence_vectors)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(sentences_name[sentence_id][:10])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

In [0]:
df_queries = pd.read_csv('keyword.txt',header = None)


In [0]:
import scipy.spatial

queries = ['美術館', 'グルメ', 'スポーツ大会', '子供がはしゃげる','ごはん']
queries = df_queries[0].values.tolist()
query_embeddings = model.encode(queries)

closest_n = 5
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_vectors, metric="cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:closest_n]:
        print(sentences_name[idx].strip(), "(Score: %.4f)" % (distance / 2))

In [80]:
%load_ext tensorboard
import os
logs_base_dir = "runs"
os.makedirs(logs_base_dir, exist_ok=True)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
import torch
from torch.utils.tensorboard import SummaryWriter

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

summary_writer = SummaryWriter()
summary_writer.add_embedding(mat=np.array(sentence_vectors), metadata=sentences_name)

In [83]:
%tensorboard --logdir {logs_base_dir}

Reusing TensorBoard on port 6006 (pid 1265), started 0:00:47 ago. (Use '!kill 1265' to kill it.)

<IPython.core.display.Javascript object>

In [0]:
!kill 1227